# Prototype for Infinite Mural

## Setup

In [ ]:
# !pip install diffusers transformers accelerate safetensors torchvision --upgrade
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 --upgrade

In [ ]:
import gc
import numpy as np
import torch

from PIL import Image, ImageFilter

from torch.cuda import empty_cache

from diffusers import AutoPipelineForInpainting, ControlNetModel
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler, UniPCMultistepScheduler
from diffusers import StableDiffusionControlNetPipeline as SDCNPipeline

In [ ]:
def clear_from_gpu(pipe):
  pipe = pipe.to("cpu")
  del pipe
  gc.collect()
  empty_cache()

## Control Net

#### [Scribble](https://huggingface.co/lllyasviel/sd-controlnet-scribble)

<img src="./imgs/scribble_00.jpg" width="200px">
<img src="./imgs/scribble_01.jpg" width="200px">

In [ ]:
controlnet = ControlNetModel.from_pretrained(
  "lllyasviel/sd-controlnet-scribble",
  # "lllyasviel/sd-controlnet-depth",
  torch_dtype=torch.float16
)

pipe = SDCNPipeline.from_pretrained(
  "runwayml/stable-diffusion-v1-5",
  controlnet=controlnet,
  safety_checker=None,
  torch_dtype=torch.float16
).to("cuda")

In [ ]:
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
im = Image.open("./imgs/scribble_00.jpg").convert("RGB")
out = pipe("person", im, num_inference_steps=20)
display(out["images"][0])

## [Inpainting](https://huggingface.co/docs/diffusers/en/using-diffusers/inpaint)

Generate parts of an image.

In [ ]:
pipe = AutoPipelineForInpainting.from_pretrained(
  #"runwayml/stable-diffusion-inpainting",
  #"stable-diffusion-v1-5/stable-diffusion-inpainting",
  "stabilityai/stable-diffusion-2-inpainting",
  torch_dtype=torch.float16,
  variant="fp16"
).to("cuda")

In [ ]:
KEEP_WIDTH = 256
KEEP_BLUR = 16
RESULT_SIZE = (1440, 512)

def create_input_background(keep_width=KEEP_WIDTH, size=RESULT_SIZE):
  img_in = Image.new("L", size)
  iw, ih = size
  img_in_pxs = [(i % iw >= keep_width) * 255 for i in range(iw * ih)]
  img_in.putdata(img_in_pxs)
  return img_in.convert("RGB")

def resize_by_height(src, height=RESULT_SIZE[1]):
  iw, ih = src.size
  nw = iw * height / ih
  return src.resize((nw, height))

def gen_mural(img, prompt, keep_width=KEEP_WIDTH, size=RESULT_SIZE, blur=KEEP_BLUR, n=4):
  img = resize_by_height(img.convert("RGB"), height=size[1])
  bgd = create_input_background(keep_width=keep_width, size=size)
  mask = create_input_background().filter(ImageFilter.GaussianBlur((blur, 0)))
  bgd_np = np.array(bgd)

  for i in range(n):
    bgd_np[:, :keep_width] = np.array(img)[:, -keep_width:]
    img_in = Image.fromarray(bgd_np)
    output = pipe(prompt=prompt, image=img_in, mask_image=mask, num_inference_steps=24, width=size[0], height=size[1])
    img_out_np = np.array(output.images[0])[:, keep_width:]
    img_out = Image.fromarray(img_out_np)
    img_out.save(f"./imgs/out_{('0'+str(i))[-2:]}.jpg")
    img = img_out


### Test

In [ ]:
img = resize_by_height(Image.open("./imgs/landscape_00.jpg").convert("RGB"))
bgd = create_input_background()
mask = create_input_background().filter(ImageFilter.GaussianBlur((KEEP_BLUR, 0)))

display(img)
display(bgd)
display(mask)

In [ ]:
bgd_np = np.array(bgd)
bgd_np[:, :KEEP_WIDTH] = np.array(img)[:, -KEEP_WIDTH:]
img_in = Image.fromarray(bgd_np)

display(img_in)

In [ ]:
prompt = "bob ross oil painting of nature landscape with trees, detailed brushstrokes, mountains, lake, rocks"
output = pipe(prompt=prompt, image=img_in, mask_image=mask, num_inference_steps=24, width=RESULT_SIZE[0], height=RESULT_SIZE[1])
display(output.images[0])

### Run

In [ ]:
img = Image.open("./imgs/landscape_00.jpg")
prompt = "bob ross oil painting of nature landscape with trees, detailed brushstrokes, mountains, lake, rocks"
prompt = "bob ross oil painting of nature landscape with garbage, floods, volcano fire, trees on fire, barren burnt trees, detailed brushstrokes, mountains, sometimes a lake and rocks"
gen_mural(img, prompt)